In [1]:
### load modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
import wntr
import wntr_utils
from decimal import Decimal

In [7]:
# Create a water network model
inp_file = 'minitown_map.inp'
results_list = []
df_new = pd.read_csv('trial_30weeks_version2.csv', index_col=0)
### load new demands
d_max = 0.0125
constant = 0.00
scaler = MinMaxScaler(feature_range=(0, d_max))
temp = scaler.fit_transform(df_new.values.reshape(-1,1))
df_scaled = pd.DataFrame(columns = df_new.columns, data = temp.reshape(df_new.shape))+constant
init_levels = pd.read_csv('minitown_initial_tank_levels.csv', index_col=0)
# Simulate 30 times, with different demand patterns
week_index = 0
last_level = float(init_levels.iloc[0].name)

In [8]:
# This code simulates 30 weeks using 30 different demand patterns and initial conditions
for week_index in range(0,30):
    wn = wntr.network.WaterNetworkModel(inp_file)
    
    # we want to do step-by-step simulation to have the same number of points as MiniCPS
    wn.options.time.duration = 900
    tank = wn.get_node('TANK')
    pump1 = wn.get_link('PUMP1')
    pump2 = wn.get_link('PUMP2')
    
    tank.init_level = last_level

    # The following lines are used to replace minitown default demand patterns
    df_check = wntr_utils.get_demand_patterns_from_nodes(wn)
    df_old = wntr_utils.get_demand_patterns_from_nodes(wn)
    
    # Substitute new demands
    juncs = df_old.columns

    juncs = juncs[(df_old.sum()>0).values].tolist() # these are the demand nodes
    assert(len(juncs)==df_scaled.shape[1]) # check if they match columns in new demands
    d_juncs = dict(zip(juncs,df_scaled.columns)) # match each demand node with new demand
    
    # remove old patterns (this seems to work, can be done better to speed it up)
    for p_name in wn.pattern_name_list:
        for j_name in wn.junction_name_list:
            try:
                wn._pattern_reg.remove_usage(p_name,(wn.name, j_name))
            except:
                pass
                # print(f'Junction {j_name} not using pattern {p_name}')  
    
    new_pat={}
    new_pat['pat1'] = df_scaled['pat1'].values[168*week_index:168*(week_index+1)-1]
    new_pat['pat2'] = df_scaled['pat2'].values[168*week_index:168*(week_index+1)-1]
    new_pat['pat3'] = df_scaled['pat3'].values[168*week_index:168*(week_index+1)-1]
    new_pat['pat4'] = df_scaled['pat4'].values[168*week_index:168*(week_index+1)-1]

    # Add the weekly pattern
    for name in df_scaled.columns:
        wn.add_pattern(name=name, pattern=new_pat[name])        
        
    # We leave this loop as it is
    for name in juncs:
        junc = wn.get_node(name)
        junc.add_demand(1,d_juncs[name]) 
    
    for name in juncs:
        junc = wn.get_node(name)
        junc.add_demand(1,d_juncs[name])     
    
    # Prepare to start a week simulation
    
    wn.options.time.duration = 900
    sim = wntr.sim.WNTRSimulator(wn,mode='PDD')
    days_simulated = 7
    iteration_limit = days_simulated*(24*3600) / wn.options.time.duration    
    iteration = 0 

    result_dict=dict.fromkeys(['name','results'])
    result_dict['name'] = "Week_" + str(week_index)
    result_dict['results'] = []
    
    node_list = list(wn.node_name_list)
    junction_list = []
    for node in node_list:
        if wn.get_node(node).node_type == 'Junction':
            junction_list.append(str(node))      

    list_header = ["iteration", "TANK_LEVEL"]
    list_header.extend(junction_list)
    another_list = ["FLOW_PUMP1", "FLOW_PUMP2", "STATUS_PUMP1", "STATUS_PUMP2"]
    list_header.extend(another_list)
    result_dict['results'].append(list_header)
    # The ouput of this simulation is a dict with two fields: the week name and a data structure that can be used to write a .csv file
    # The data structure follows the same pattern as MiniCPS for ease of comparation
    while iteration <= iteration_limit:
        
        results = sim.run_sim()        
        values_list = []
        values_list.extend([iteration, tank.level])
        
        for junction in junction_list:
            values_list.extend([wn.get_node(junction).head - wn.get_node(junction).elevation])            
            
        values_list.extend([pump1.flow, pump2.flow])

        if type(pump1.status) is int:
            values_list.extend([pump1.status])
        else:
            values_list.extend([pump1.status.value])

        if type(pump2.status) is int:
            values_list.extend([pump2.status])
        else:
            values_list.extend([pump2.status.value])            

        result_dict['results'].append(values_list)                        
        iteration += 1
                            
    # we finished a week simulation, move on
    results_list.append(result_dict)    
    week_index += 1
                            
    # update tank level for next week simulation
    if week_index < 30:
        last_level = float(init_levels.iloc[week_index].name)

C:\Users\Azulm\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "minitown_map.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


IndexError: single positional indexer is out-of-bounds

In [14]:
for week_index in range(0,30):
    filename = 'wntr_only_results/minitown_wntr_week_' + str(week_index) + '.csv'
    with open(filename, 'w', newline='\n') as f:
        writer = csv.writer(f)
        writer.writerows(results_list[week_index]['results'])

In [ ]:
nr = 15 
nc = 2

f, ax = plt.subplots(nr,nc,figsize=(30,150))
week_index = 0

for row_index in range(0,15):
    for col_index in range(0,2):    
        week_wntr_results = pd.read_csv('wntr_only_results/minitown_wntr_week_' + str(week_index) + '.csv')
        week_minicps_results = pd.read_csv('../ICS_topologies/minitown_topology_dataset_generation/data/week_' + str(week_index) + '/physical_process.csv')
        ax[row_index][col_index].plot(week_wntr_results["TANK_LEVEL"], label='WNTR')
        ax[row_index][col_index].plot(week_minicps_results["TANK_LEVEL"], label='MiniCPS')
        ax[row_index][col_index].set_title("Week " + str(week_index))
        ax[row_index][col_index].legend()
        ax[row_index][col_index].grid()
        week_index += 1
        
f, ax_2 = plt.subplots(1,1)
for week_index in range(0,30):
    week_wntr_results = pd.read_csv('wntr_only_results/minitown_wntr_week_' + str(week_index) + '.csv')
    ax_2.plot(week_wntr_results["TANK_LEVEL"])
    ax_2.set_title("All simulations together WNTR")
    
f, ax_2 = plt.subplots(1,1)
for week_index in range(0,30):
    week_minicps_results = pd.read_csv('../ICS_topologies/minitown_topology_dataset_generation/data/week_' + str(week_index) + '/physical_process.csv')
    ax_2.plot(week_minicps_results["TANK_LEVEL"])
    ax_2.set_title("All simulations together MiniCPS")    